In [2]:
import numpy as np
import pandas as pd
import webbrowser as wb
from bs4 import BeautifulSoup
import requests

from pandas import Series, DataFrame
from numpy.random import randn

countries = DataFrame(['Switzerland', 'Netherlands','United States', 'Canada', 'Australia', 'New Zealand', 'Norway', 'United Kingdom', 
                            'Germany', 'Belgium', 'Sweden', 'France', 'Italy', 'Denmark', 'Finland', 'Turkey', 'Iran'], columns = ['Country'])

link = 'https://en.wikipedia.org/wiki/{0}'
country_list_links = Series(['List_of_countries_by_Human_Development_Index', 'List_of_countries_by_income_equality','List_of_countries_by_GDP_(PPP)_per_capita','List_of_countries_by_real_GDP_growth_rate'
                            ,'List_of_countries_by_GDP_(nominal)_per_capita','List_of_countries_and_dependencies_by_population','List_of_countries_and_dependencies_by_population','List_of_countries_by_average_wage'
                            ,'Satisfaction_with_Life_Index','Happy_Planet_Index','List_of_countries_by_unemployment_rate','List_of_countries_by_job_security','List_of_countries_by_inflation_rate'
                        ,'Ease_of_doing_business_index','Index_of_Economic_Freedom','List_of_countries_by_tax_rates','List_of_minimum_wages_by_country','List_of_countries_by_past_and_projected_GDP_(nominal)_per_capita',
                             'List_of_countries_by_GDP_growth_1980–2010','List_of_countries_by_GDP_(real)_per_capita_growth_rate','Global_Peace_Index'])
links = []
for i in range(len(country_list_links)):
    links.append(link.format(country_list_links[i]))

In [3]:
def getRawTables(link):
    """extract tables of a page"""
    # Request content from web page
    result = requests.get(link)
    c = result.content

    # Set as Beautiful Soup Object
    soup = BeautifulSoup(c)
    #print(soup)
    rawTables = soup.findAll("table",{'class':'wikitable'})
    print(len(rawTables))
    return rawTables

In [4]:
def getRawDataFromTable(table):
    """extract raw datas of cells in one table"""
    cells=table.findAll("td")
    
    data = []
    for i in range(len(cells)):
        link = cells[i].findAll("a")
        if len(link) > 0:
            data.append(link[0].text)
        else:
            data.append(cells[i].text)
            
    return data

In [5]:
def getColumnsFromTable(table):
    """extract column names for each table"""
    temp = table.findAll("th")
    
    columns = []
    for i in range(len(temp)):
        columns.append(temp[i].text)
        
    return columns

In [6]:
def formDataFrame(table):
    """function that gets table and call to other function to form a dataframe"""
    rawData = Series(getRawDataFromTable(table))
    columns = getColumnsFromTable(table)
    columnCount = len(columns)
    rowCount = len(rawData)/columnCount
    
    res = DataFrame(rawData.reshape(rowCount, columnCount), columns = columns)
    
    return res

In [7]:
def extractTables(link):
    tables = getRawTables(link)
    
    dfTables = []
    
    #dfTables.append(formDataFrame(tables[0]))
    
    for i in range(len(tables)):
            dfTables.append(formDataFrame(tables[i]))
            
    return dfTables

In [8]:
def desiredCountriesData(partLink):
    """extract data for those countries that I'm looking for"""
    link = 'https://en.wikipedia.org/wiki/{0}'
    countries = DataFrame(index=['Switzerland', 'Netherlands','United States', 'Canada', 'Australia', 'New Zealand', 'Norway', 'United Kingdom', 
                            'Germany', 'Belgium', 'Sweden', 'France', 'Italy', 'Denmark', 'Finland', 'Turkey', 'Iran'])
    
    link = link.format(partLink)
    print(link)
    wb.open(link)
    tables = extractTables(link)
    
    res = []
    for i in range(len(tables)):
        res.append(tables[i][tables[i]['Country'].isin(countries.index)])
    
    
    return res

In [44]:
#[0, 1, 3, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
#[2, 4, 7, 9, 12]
res = desiredCountriesData(country_list_links[20])
for i in range(len(res)):
    print('{0}_{1}'.format(country_list_links[1], i))
    print(res[i])

https://en.wikipedia.org/wiki/Global_Peace_Index
3


ValueError: total size of new array must be unchanged

In [35]:
temp = res[0]
countries = countries.merge(temp, on='Country', how='outer')

In [45]:
countries.columns = ['Country','GDP PPP','GDP Nominal','Average Wage PPP','Complusory Deduction','HPI','Life Expectancy','Ecological Footprint','inflation','drop']

In [47]:
countries = countries.drop(['drop'],1)

In [52]:
countries.drop('inflation',1).to_csv('countries2.csv',index=False)